In [1]:
import feedparser, requests, json, time
from apscheduler.schedulers.blocking import BlockingScheduler

In [29]:
FLOW_URL = r"https://default723a5a87f39a4a2292473fc240c013.96.environment.api.powerplatform.com:443/powerautomate/automations/direct/workflows/90fa284934f1486f806f96b119ad7155/triggers/manual/paths/invoke?api-version=1&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=SUQbG59Bee34wkZ_gs9yReKiPp9NblwXmQYvDVR--jg" 
KEYWORDS = ["qudits"]
FILTER_MODE = False  # True = only post matching keywords, False = post every new paper
MAX_RESULTS = 1

In [35]:

def fetch_arxiv():
    url = f"http://export.arxiv.org/api/query?search_query=all&sortBy=submittedDate&sortOrder=descending&max_results={MAX_RESULTS}"
    feed = feedparser.parse(url)
    print(feed)
    return feed.entries

def match_keywords(title, summary):
    text = (title + " " + summary).lower()
    return any(k.lower() in text for k in KEYWORDS)

def post_to_teams(title, link, summary, authors):
    payload = {
        "type": "message",
        "attachments": [
            {
                "contentType": "application/vnd.microsoft.card.adaptive",
                "content": {
                    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                    "type": "AdaptiveCard",
                    "version": "1.5",
                    "body": [
                        {
                            "type": "Container",
                            "items": [
                                {
                                    "type": "TextBlock",
                                    "text": title,
                                    "wrap": True,
                                    "weight": "Bolder",
                                    "size": "Large"
                                },
                                {
                                    "type": "TextBlock",
                                    "text": f"👨‍🔬 **Authors:** {authors}",
                                    "wrap": True,
                                    "spacing": "Small"
                                },
                                {
                                    "type": "TextBlock",
                                    "text": summary,
                                    "wrap": True,
                                    "spacing": "Medium"
                                }
                            ],
                            "style": "emphasis"
                        }
                    ],
                    "actions": [
                        {
                            "type": "Action.OpenUrl",
                            "title": "🔗 Read on arXiv",
                            "url": link
                        }
                    ]
                }
            }
        ]
    }

    r = requests.post(FLOW_URL, json=payload)
    print(f"→ Posted: {title} | HTTP {r.status_code}")

def main():
    entries = fetch_arxiv()
    if not entries:
        print("No papers retrieved from arXiv.")
        return

    matched = []
    for e in entries:
        if not FILTER_MODE or match_keywords(e.title, e.summary):
            matched.append(e)

    if matched:
        print(f"Found {len(matched)} new paper(s) matching criteria:")
        for e in matched:
            authors = ', '.join(a.name for a in e.authors) if hasattr(e, "authors") else "Unknown"
            print(f"- {e.title} | Authors: {authors}")
            post_to_teams(e.title, e.link, e.summary, authors)
    else:
        print("No new papers matched the criteria.")

if __name__ == "__main__":
    main()

{'bozo': False, 'entries': [{'id': 'http://arxiv.org/abs/2511.02831v1', 'guidislink': True, 'link': 'http://arxiv.org/abs/2511.02831v1', 'updated': '2025-11-04T18:58:20Z', 'updated_parsed': time.struct_time(tm_year=2025, tm_mon=11, tm_mday=4, tm_hour=18, tm_min=58, tm_sec=20, tm_wday=1, tm_yday=308, tm_isdst=0), 'published': '2025-11-04T18:58:20Z', 'published_parsed': time.struct_time(tm_year=2025, tm_mon=11, tm_mday=4, tm_hour=18, tm_min=58, tm_sec=20, tm_wday=1, tm_yday=308, tm_isdst=0), 'title': 'GeoCrossBench: Cross-Band Generalization for Remote Sensing', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://export.arxiv.org/api/query?search_query=all&sortBy=submittedDate&sortOrder=descending&max_results=1', 'value': 'GeoCrossBench: Cross-Band Generalization for Remote Sensing'}, 'summary': 'The number and diversity of remote sensing satellites grows over time, while\nthe vast majority of labeled data comes from older satellites. As the\nfoundation models for E

In [8]:
scheduler = BlockingScheduler()

In [9]:
scheduler.add_job(check_and_post, "interval", hours=3)

<Job (id=a4adc670b1604605a009d29c5e726c08 name=check_and_post)>

In [ ]:
scheduler.start()

In [19]:
import feedparser, json, datetime, requests
import time
FLOW_URL = r"https://default723a5a87f39a4a2292473fc240c013.96.environment.api.powerplatform.com:443/powerautomate/automations/direct/workflows/90fa284934f1486f806f96b119ad7155/triggers/manual/paths/invoke?api-version=1&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=SUQbG59Bee34wkZ_gs9yReKiPp9NblwXmQYvDVR--jg" 

SEARCH_URL = (
    "http://export.arxiv.org/api/query?"
    "search_query=(ti:(qudit+OR+trapped+ions)+OR+abs:(qudit+AND+ion))"
    "&sortBy=submittedDate&sortOrder=descending&max_results=50"
)

SEARCH_URL = (
    "http://export.arxiv.org/api/query?"
    "search_query=(ti:(Trapped+ion)+OR+abs:(qudit+AND+Trapped+ion))"
    "&sortBy=submittedDate&sortOrder=descending&max_results=50"
)

SEEN_FILE = "seen.json"


def load_seen():
    try:
        with open(SEEN_FILE, "r") as f:
            return set(json.load(f))
    except:
        return set()


def save_seen(seen):
    with open(SEEN_FILE, "w") as f:
        json.dump(list(seen), f)


def fetch_latest():
    feed = feedparser.parse(SEARCH_URL)
    if not feed.entries:
        print("⚠️ No entries found.")
        return []
    return feed.entries


def check_new_papers():
    seen = load_seen()
    feed_entries = fetch_latest()
    new_papers = []

    for entry in feed_entries:
        if entry.id not in seen:
            seen.add(entry.id)
            new_papers.append(entry)

    save_seen(seen)
    return new_papers


def post_to_teams(title, link, summary, authors, date):
    payload = {
        "type": "message",
        "attachments": [
            {
                "contentType": "application/vnd.microsoft.card.adaptive",
                "content": {
                    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                    "type": "AdaptiveCard",
                    "version": "1.5",
                    "body": [
                        {
                            "type": "Container",
                            "items": [
                                {
                                    "type": "TextBlock",
                                    "text": title,
                                    "wrap": True,
                                    "weight": "Bolder",
                                    "size": "Large",
                                },
                                {
                                    "type": "TextBlock",
                                    "text": f"👨‍🔬 **Authors:** {authors}",
                                    "wrap": True,
                                    "spacing": "Small",
                                },
                                {
                                    "type": "TextBlock",
                                    "text": f"📅 **Published:** {date}",
                                    "wrap": True,
                                    "spacing": "Small",
                                },
                                {
                                    "type": "TextBlock",
                                    "text": "📝 **Abstract:**",
                                    "wrap": True,
                                    "weight": "Bolder"
                                },
                                {
                                    "type": "TextBlock",
                                    "text": summary,
                                    "wrap": True,
                                    "spacing": "Medium",
                                },
                            ],
                            "style": "emphasis",
                        }
                    ],
                    "actions": [
                        {
                            "type": "Action.OpenUrl",
                            "title": "🔗 Read on arXiv",
                            "url": link,
                        }
                    ],
                },
            }
        ],
    }

    r = requests.post(FLOW_URL, json=payload)
    print(f"→ Posted: {title} | HTTP {r.status_code}")


def summarize(entry):
    title = entry.title.strip()
    link = entry.link
    authors = ", ".join(a.name for a in entry.authors)
    date = entry.published.split("T")[0]
    summary = entry.summary.replace("\n", " ")
    return title, link, summary, authors, date


def main():
    print(f"\n🕓 Checking arXiv for new 'qudit trapped ion' papers (title+abstract)... ({datetime.date.today()})")
    new_papers = check_new_papers()

    if new_papers:
        print(f"✅ Found {len(new_papers)} new paper(s):\n")
        for e in new_papers[:]:
            title, link, summary, authors, date = summarize(e)
            print(f"- {title} | {authors} | {date}")
            # post_to_teams(title, link, summary, authors, date)
            # print("⏳ Waiting 5 minute before next post...")
            # time.sleep(5)
    else:
        print("No new papers since last check.")


if __name__ == "__main__":
    main()



🕓 Checking arXiv for new 'qudit trapped ion' papers (title+abstract)... (2025-11-12)
✅ Found 50 new paper(s):

- Practical trapped-ion protocols for universal qudit-based quantum computing | Pei Jiang Low, Brendan M. White, Andrew A. Cox, Matthew L. Day, Crystal Senko | 2019-07-19
- Universal quantum computing with qubits embedded in trapped-ion qudits | Anastasiia S. Nikolaeva, Evgeniy O. Kiktenko, Aleksey K. Fedorov | 2023-02-06
- Control and Readout of a 13-level Trapped Ion Qudit | Pei Jiang Low, Brendan White, Crystal Senko | 2023-06-06
- Native qudit entanglement in a trapped ion quantum processor | Pavel Hrmo, Benjamin Wilhelm, Lukas Gerster, Martin W. van Mourik, Marcus Huber, Rainer Blatt, Philipp Schindler, Thomas Monz, Martin Ringbauer | 2022-06-08
- Efficient Implementation of a Quantum Algorithm with a Trapped Ion Qudit | Xiaoyang Shi, Jasmine Sinanan-Singh, Timothy J. Burke, John Chiaverini, Isaac L. Chuang | 2025-06-11
- Realizing quantum gates with optically-addressabl

In [16]:
#!/usr/bin/env python3
 
import feedparser

import datetime

from datetime import timezone
 
SEARCH_URL = (

    "http://export.arxiv.org/api/query?"

    "search_query=(ti:(qudit+OR+trapped+ion)+OR+abs:(qudit+OR+trapped+ion))"

    "&sortBy=submittedDate&sortOrder=descending&max_results=100"

)
 
def fetch_papers():

    """Fetch papers from arXiv"""

    print(f"Fetching from: {SEARCH_URL}\n")

    feed = feedparser.parse(SEARCH_URL)

    if not feed.entries:

        print("⚠️ No entries found.")

        return []

    return feed.entries
 
def filter_by_date(entries, days=5):

    """Keep only papers from the past N days"""

    cutoff = datetime.datetime.now(timezone.utc) - datetime.timedelta(days=days)

    filtered = []

    for entry in entries:

        # Parse published date (ISO format with Z timezone)

        published = datetime.datetime.fromisoformat(entry.published.replace('Z', '+00:00'))

        if published > cutoff:

            filtered.append(entry)

    return filtered
 
def summarize(entry):

    """Extract paper metadata"""

    title = entry.title.strip()

    link = entry.link

    authors = ", ".join(a.name for a in entry.authors)

    date = entry.published.split("T")[0]

    return title, link, authors, date
 
def main():

    print(f"\n🕓 Checking arXiv for 'qudit' or 'trapped ion' papers from past 5 days...")

    print(f"Today: {datetime.date.today()}\n")

    # Fetch all papers matching query

    all_papers = fetch_papers()

    print(f"Total matches: {len(all_papers)}")

    # Filter to only past 5 days

    recent_papers = filter_by_date(all_papers, days=5)

    print(f"Papers from past 5 days: {len(recent_papers)}\n")

    if recent_papers:

        print(f"{'Date':<12} {'Title':<70} {'Authors'}")

        print("-" * 120)

        for entry in recent_papers:

            title, link, authors, date = summarize(entry)

            # Truncate long titles

            title_short = title[:67] + "..." if len(title) > 70 else title

            print(f"{date}  {title_short:<70} {authors[:30]}")

    else:

        print("No papers found from past 5 days.")
 
if __name__ == "__main__":

    main()

 


🕓 Checking arXiv for 'qudit' or 'trapped ion' papers from past 5 days...
Today: 2025-11-12

Fetching from: http://export.arxiv.org/api/query?search_query=(ti:(qudit+OR+trapped+ion)+OR+abs:(qudit+OR+trapped+ion))&sortBy=submittedDate&sortOrder=descending&max_results=100

Total matches: 100
Papers from past 5 days: 0

No papers found from past 5 days.
